In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

In [2]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int4", torch_dtype="auto", device_map="auto"
)

/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.
`Q

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int4")

### Data

In [4]:
import pandas as pd
from pathlib import Path
from PIL import Image

In [5]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/dataset_files/comics_pg_w_images.csv", index_col=0)

In [6]:
df

,file_name,page_nr,split,utterance,emotion_c,comics_title,comics_id,image_path
0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,TRAIN,"[""THIS VILE THING ATTACKED THE SMALL BEASTS OF...","[['anger'], ['anger'], ['fear'], ['fear'], ['f...",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...
1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,TRAIN,"[""NO-- #GKKK\u2026#"", ""#CHOMP!"", ""BY THE SKIN...","[['fear'], ['anger'], ['surprise'], ['anger'],...",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...
2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,TRAIN,"[""COME ON, BEAST!"", ""SHOW YOURSELF!"", ""WHY DO ...","[['joy'], ['joy'], ['anger'], ['anger']]",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...
3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,TRAIN,"[""#AARGH! ""]","[['fear', 'surprise']]",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...
4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,TRAIN,"[""I, THE GREEN TORCH, HAVE BEEN TASKED WITH PR...","[['anger'], ['anger'], ['fear'], ['fear', 'sur...",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...
...,...,...,...,...,...,...,...,...
869,QC copy - 2200 - Stillwater 13.xlsx,16,TEST,"[""WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO...","[['anger'], ['anger'], ['anger'], ['anger'], [...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...
870,QC copy - 2200 - Stillwater 13.xlsx,17,TEST,"[""SO WHAT ARE WE GOING TO DO?"", ""THE WAY I SEE...","[['sadness', 'surprise'], ['anger'], ['anger']...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...
871,QC copy - 2200 - Stillwater 13.xlsx,18,TEST,"[""KIDDIE COUNCIL'S BEEN GOING A LONG TIME... ""...","[['anger', 'sadness'], ['anger'], ['anger'], [...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...
872,QC copy - 2200 - Stillwater 13.xlsx,19,TEST,"[""IT'S BEEN\u2026 PEACEFUL. ASIDE FROM SHIT LI...","[['anger'], ['joy'], ['joy'], ['anger', 'surpr...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...


In [7]:
def build_generation_instruction(comics_title, page_utterances):
   
    emotion_classes = ["anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"]
    formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
    instruction = f"""### Emotion Analysis for Comics

You are an emotion analysis expert for comic dialogue. Your task is to analyze utterances based on the immediate context.

INPUT:
- List of utterances from a page in a comic
- An image of the comics page

OUTPUT:
- JSON with single key "emotions"
- Value: array of emotion arrays matching utterance order
- ONLY use these emotions: {formatted_classes}
- NO OTHER EMOTION LABELS ARE ALLOWED

RULES:
1. Each utterance must have at least one emotion from the list above
2. Multiple emotions per utterance are allowed
3. Keep emotions in arrays even for single emotions
4. Maintain exact emotion spelling and case
5. No explanations, only JSON output

Example format:
{{"emotions": [["joy"], ["anger", "fear"], ["neutral"]]}}

Comic Information:
Title: {comics_title}

Utterances to Classify:
{page_utterances}

"""
    return instruction

In [8]:
def build_image_modality(image_path):
    
    return Image.open(image_path)

In [9]:
def convert_to_conversation(row):
  
    comics_title = row.comics_title
    image_path = row.image_path
    labels = row.emotion_c
    
    utterances_l = eval(row['utterance'])
    pg_utterances = "\n".join(f"{i+1}. {title}" for i, title in enumerate(utterances_l))
  
    instruction = build_generation_instruction(comics_title, pg_utterances)
    image = build_image_modality(image_path)
    
    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : image} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : ""} ]
        },
    ]
    return { "messages" : conversation }
pass

In [10]:
# Apply the function to each row and store the results in a list
comics_mm_dataset = [convert_to_conversation(row) for _, row in df.iterrows()]

In [11]:
len(comics_mm_dataset)

874

In [12]:
#comics_mm_dataset = comics_mm_dataset[:len(comics_mm_dataset)/2]

In [13]:
comics_mm_dataset[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': '### Emotion Analysis for Comics\n\nYou are an emotion analysis expert for comic dialogue. Your task is to analyze utterances based on the immediate context.\n\nINPUT:\n- List of utterances from a page in a comic\n- An image of the comics page\n\nOUTPUT:\n- JSON with single key "emotions"\n- Value: array of emotion arrays matching utterance order\n- ONLY use these emotions: "anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"\n- NO OTHER EMOTION LABELS ARE ALLOWED\n\nRULES:\n1. Each utterance must have at least one emotion from the list above\n2. Multiple emotions per utterance are allowed\n3. Keep emotions in arrays even for single emotions\n4. Maintain exact emotion spelling and case\n5. No explanations, only JSON output\n\nExample format:\n{"emotions": [["joy"], ["anger", "fear"], ["neutral"]]}\n\nComic Information:\nTitle: Jurassic League #4\n\nUtterances to Classify:\n1. THIS VILE THING ATTA

In [14]:
from PIL import Image

def resize_images(images, max_size=512):
    resized_images = []
    for img in images:
        if isinstance(img, Image.Image):  # Check if it's a PIL Image
            img = img.resize((max_size, max_size))#, Image.ANTIALIAS) # type: ignore
        resized_images.append(img)
    return resized_images


In [15]:
#comics_mm_dataset[0]['messages'][0]['content'][1]['image']

In [16]:
from tqdm import tqdm

In [19]:
BATCH_SIZE = 8

In [20]:
# Store outputs
raw_outputs = []

# Split the dataset into batches
for i in tqdm(range(0, len(comics_mm_dataset), BATCH_SIZE)):
    batch = comics_mm_dataset[i:i + BATCH_SIZE]

    # Prepare batch inputs
    batch_texts = []
    batch_images = []
    
    for message in batch:
        # Process text and images for the current message
        text = processor.apply_chat_template(
            message['messages'], tokenize=False, add_generation_prompt=True
        )
        image_inputs, _ = process_vision_info(message['messages'])
        image_inputs = resize_images(image_inputs, max_size=512)

        # Append to batch lists
        batch_texts.append(text)
        batch_images.append(image_inputs)
    
    # Process the batch with the processor
    inputs = processor(
        text=batch_texts,
        images=batch_images,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate outputs for the batch
    generated_ids = model.generate(**inputs, max_new_tokens=256)
    
    # Append outputs
    raw_outputs.extend(generated_ids)

# Final outputs are stored in raw_outputs


100%|██████████| 110/110 [32:09<00:00, 17.54s/it]


In [29]:
len(raw_outputs)

874

In [30]:
op = [processor.decode(x).split("<|im_start|>assistant\n<|im_end|>\n<|im_start|>assistant\n")[1].split("<|im_end|>")[0] for x in raw_outputs]

In [31]:
op

['```json\n{\n  "emotions": [\n    ["anger"],\n    ["sadness", "disgust"],\n    ["joy"],\n    ["sadness"],\n    ["anger"],\n    ["sadness"],\n    ["anger", "fear"],\n    ["surprise"],\n    ["anger", "fear"],\n    ["anger"]\n  ]\n}\n```',
 '```json\n{\n  "emotions": [\n    ["neutral"],\n    ["anger", "disgust"],\n    ["anger", "fear"],\n    ["anger", "sadness"],\n    ["anger", "joy"],\n    ["anger", "sadness"],\n    ["neutral"],\n    ["anger", "sadness"],\n    ["anger", "joy"]\n  ]\n}\n```',
 '{"emotions": [["anger"], ["anger", "fear"], ["anger", "disgust"], ["anger", "joy"]]}',
 '{"emotions": [["anger"], ["fear", "sadness"], ["neutral"]]}',
 '{"emotions": [["anger"], ["joy", "anger"], ["fear"], ["sadness"], ["neutral"], ["neutral"]]}',
 '{"emotions": [["anger"], ["fear"], ["sadness"], ["neutral"], ["joy"], ["neutral"]]}',
 '{"emotions": [["anger"], ["sadness"], ["neutral"], ["anger"], ["neutral"]]}',
 '```json\n{\n  "emotions": [\n    ["neutral"],\n    ["anger", "fear"],\n    ["neutral

In [32]:
import json_repair

In [33]:
preds = [json_repair.loads(x) for x in op]

In [35]:
preds = [x['emotions'] for x in preds]

In [36]:
preds

[[['anger'],
  ['sadness', 'disgust'],
  ['joy'],
  ['sadness'],
  ['anger'],
  ['sadness'],
  ['anger', 'fear'],
  ['surprise'],
  ['anger', 'fear'],
  ['anger']],
 [['neutral'],
  ['anger', 'disgust'],
  ['anger', 'fear'],
  ['anger', 'sadness'],
  ['anger', 'joy'],
  ['anger', 'sadness'],
  ['neutral'],
  ['anger', 'sadness'],
  ['anger', 'joy']],
 [['anger'], ['anger', 'fear'], ['anger', 'disgust'], ['anger', 'joy']],
 [['anger'], ['fear', 'sadness'], ['neutral']],
 [['anger'],
  ['joy', 'anger'],
  ['fear'],
  ['sadness'],
  ['neutral'],
  ['neutral']],
 [['anger'], ['fear'], ['sadness'], ['neutral'], ['joy'], ['neutral']],
 [['anger'], ['sadness'], ['neutral'], ['anger'], ['neutral']],
 [['neutral'],
  ['anger', 'fear'],
  ['neutral'],
  ['neutral'],
  ['anger', 'fear'],
  ['neutral'],
  ['neutral'],
  ['neutral'],
  ['anger', 'fear'],
  ['neutral'],
  ['anger', 'fear'],
  ['neutral']],
 [['neutral'],
  ['anger', 'fear'],
  ['neutral'],
  ['sadness'],
  ['fear'],
  ['neutral'],
 

In [37]:
grounds = df.emotion_c.tolist()

In [38]:
len(grounds)

874

In [39]:
len(preds)

874

In [41]:
import ast

grounds = [ast.literal_eval(x) for x in grounds]

In [43]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds, preds)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

3 1 3
5 10 6
8 14 12
12 13 12
14 11 10
17 1 2
26 14 13
30 4 6
33 11 10
39 2 3
42 13 12
46 11 10
47 11 10
51 1 2
52 11 10
53 1 2
55 1 3
60 8 7
73 15 14
74 5 7
75 18 17
76 16 15
77 16 12
81 1 2
84 11 10
91 1 2
93 11 10
95 8 7
99 11 10
101 16 14
102 21 20
106 16 14
109 14 13
110 4 3
111 13 12
113 23 28
114 23 21
115 13 12
116 12 11
118 14 13
119 10 9
120 11 10
123 13 12
124 13 12
125 13 12
129 13 12
134 2 3
140 13 12
144 2 3
148 12 11
153 13 12
154 8 7
160 13 12
161 18 16
167 13 12
172 11 10
173 14 13
179 9 10
183 17 16
185 1 2
186 2 3
188 5 6
192 13 12
193 11 10
200 18 15
205 22 60
206 11 10
214 11 10
216 15 14
226 11 10
230 15 14
232 17 16
233 13 12
241 11 10
242 12 10
245 13 12
247 16 15
248 15 14
249 17 16
250 14 12
253 16 15
254 17 32
255 14 13
265 1 2
270 1 2
272 19 59
273 19 61
274 23 59
275 13 12
282 11 10
283 25 49
286 14 13
291 17 16
292 15 13
296 9 8
301 31 59
302 11 10
306 13 12
309 11 10
312 16 14
318 13 12
319 12 11
320 19 16
321 17 60
322 11 10
323 14 13
325 14 13
331 2 3
3

In [44]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    del preds[idx]

In [45]:
len(grounds), len(preds)

(644, 644)

In [46]:
grounds = [item for sublist in grounds for item in sublist]
predictions = [item for sublist in preds for item in sublist]

In [47]:
len(grounds), len(predictions)

(4715, 4715)

In [49]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

In [50]:
mlb = MultiLabelBinarizer()

In [51]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['embarrassment', 'excitement', 'worry'] will be ignored
  warnings.warn(


In [52]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=mlb.classes_, digits=3))

              precision    recall  f1-score   support

       anger      0.543     0.425     0.477      1609
     disgust      0.167     0.262     0.204       225
        fear      0.387     0.200     0.264      1196
         joy      0.524     0.186     0.274      1070
     neutral      0.087     0.720     0.155       279
     sadness      0.464     0.393     0.426      1191
    surprise      0.667     0.021     0.040      1264

   micro avg      0.314     0.275     0.293      6834
   macro avg      0.406     0.315     0.263      6834
weighted avg      0.491     0.275     0.296      6834
 samples avg      0.293     0.293     0.278      6834



/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# raw_outputs = []

# for message in tqdm(comics_mm_dataset):
    
#     text = processor.apply_chat_template(
#     message['messages'], tokenize=False, add_generation_prompt=True)

#     image_inputs, _ = process_vision_info(message['messages'])
#     image_inputs = resize_images(image_inputs, max_size=1024)
    
#     inputs = processor(
#     text=text,
#     images=image_inputs,
#     padding=True,
#     return_tensors="pt",)

#     inputs = inputs.to("cuda")

#     generated_ids = model.generate(**inputs, max_new_tokens=256)
#     raw_outputs.append(generated_ids)

In [42]:
# raw_outputs = []

# for message in tqdm(comics_mm_dataset):
    
#     #print(len(message))
#     #print(message['messages'])
    
#     text = processor.apply_chat_template(
#     message['messages'], tokenize=False, add_generation_prompt=True
# )
#     #print(text)
#     #break
#     #image = comics_mm_dataset[0]['messages'][0]['content'][1]['image']
#     image_inputs, _ = process_vision_info(message['messages'])
#     image_inputs = resize_images(image_inputs, max_size=1024)
#     #image_inputs, _ = process_vision_info(message['messages'])
#     #print(image_inputs)
    
#     inputs = processor(
#     #text=[text],
#     text=text,
#     images=image_inputs,
#     #videos=video_inputs,
#     padding=True,
#     return_tensors="pt",
# )
#     #print(inputs)
#     #break
#     inputs = inputs.to("cuda")
#     #print(inputs)
#     num_elements = inputs.pixel_values.numel()

#     # Get the size of each element in bytes
#     bytes_per_element = inputs.pixel_values.element_size()

#     # Calculate total memory in bytes
#     memory_size = num_elements * bytes_per_element
#     print(f"Total memory size: {memory_size / (1024 ** 2):.4f} MB")

#     #break
#     generated_ids = model.generate(**inputs, max_new_tokens=256)
#     raw_outputs.append(generated_ids)
    
    
    
# #     input_text = tokenizer.apply_chat_template(message['messages'], add_generation_prompt = True)
# #     image = message['messages'][0]['content'][1]['image']
# #     #print(input_text)
# #     #break
   
# #     inputs = tokenizer(
# #     image,
# #     input_text,
# #     add_special_tokens = False,
# #     return_tensors = "pt",
# # ).to("cuda")
# #     #print(inputs['input_ids'])
# #     #print(tokenizer.decode(inputs['input_ids'][0]))
# #     #break
    
# #     #output = model.generate(input_ids=inputs, max_new_tokens=128)[0]
# #     output = model.generate(**inputs, max_new_tokens=512)[0]
    
# #     #input_length = inputs.shape[1]
# #     #generated_tokens = output[input_length:]
    
# #     #decoded_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)  
# #     decoded_output = tokenizer.decode(output, skip_special_tokens=True)
# #     raw_outputs.append(decoded_output)

  0%|          | 0/874 [00:00<?, ?it/s]

Total memory size: 24.5658 MB


  0%|          | 1/874 [00:09<2:18:44,  9.54s/it]

Total memory size: 24.5658 MB


  0%|          | 2/874 [00:15<1:49:25,  7.53s/it]

...

Total memory size: 24.5658 MB


  1%|▏         | 13/874 [01:31<1:59:39,  8.34s/it]

Total memory size: 24.5658 MB


  1%|▏         | 13/874 [01:34<1:44:37,  7.29s/it]


KeyboardInterrupt: 

In [25]:
#inputs.pixel_values.shape

In [26]:
#!nvidia-smi